### Instalando pacotes

In [3]:

!pip install openpyxl
!pip install pandas
!pip install numpy

In [4]:
import pandas as pd
import numpy as np
import openpyxl

# Avaliação 03
Grupo 5:
*   Allane
*   Item da lista



### Instanciando os grafos
Estamos usando DataFrames da biblioteca pandas para representar as matrizes de adjacencia dos grafos.

In [5]:
matriz_problema_km = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Km")
matriz_problema_min = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Min")
cidades = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Cidades")

### Estrutura da Busca Local

In [6]:
# Funca para obter o custo total a partir de uma solucao (caminho)
def calcular_custo(solucao, grafo):
    if len(solucao) <= 1:
        return
    custo = 0
    for prox in range(1, len(solucao)):
        v1 = solucao[prox - 1]
        v2 = solucao[prox]
        custo += grafo[v1][v2 - 1]
    return custo

# O argumento 'heuristica' eh o algoritmo da busca local que sera realizada (shift, swap etc).
# ele deve retornar uma vizinhanca - a lista de solucoes geradas
def busca_local_primeira_melhoria(solucao, custo_original, heuristica, grafo):
    custo_atual = custo_original
    solucao_atual = solucao
    melhoria = True
    while melhoria:
        melhoria = False
        for nova_solucao in heuristica(solucao_atual):
            novo_custo = calcular_custo(nova_solucao, grafo)
            if novo_custo < custo_atual:
                solucao_atual = nova_solucao
                custo_atual = novo_custo
                melhoria = True
                break
    return solucao_atual, custo_atual


### Heurísticas

In [7]:
def shift(solucao : list):
    solucao_cp = solucao[:]
    for idx_vert_shift in range(1, len(solucao) - 1):
        for idx_destino_shift in range(2, len(solucao) - 1):
            vert_shift = solucao_cp.pop(idx_vert_shift)
            solucao_cp.insert(idx_destino_shift, vert_shift)
            yield solucao_cp

def swap(solucao : list):
    solucao_cp = solucao[:]
    for idx_vert_swap in range(1, len(solucao) - 1):
        for idx_destino_swap in range(2, len(solucao) - 1):
            vert_swap = solucao_cp[idx_vert_swap]
            solucao_cp[idx_vert_swap] = solucao_cp[idx_destino_swap]
            solucao_cp[idx_destino_swap] = vert_swap
            # print("Solucao do swap : ", solucao_cp)
            yield solucao_cp

def inversao(solucao):
    solucao_cp = solucao[:]
    for idx_vert_inicio in range(1, len(solucao) - 1):
        for idx_destino in range(2, len(solucao) - 1):
            sublista_invertida = solucao_cp[idx_vert_inicio:idx_destino]
            sublista_invertida.reverse()
            yield solucao_cp[0:idx_vert_inicio] + sublista_invertida + solucao_cp[idx_destino:]


## Busca por vizinho mais próximo

In [8]:
# Estamos considerando o label numerico
def vizinho_mais_proximo(grafo : pd.DataFrame, vertice_inicial : int | str):
    # Criando conjunto de vertices nao visitados
    # o conjunto contem os indices referentes ao vertice na matriz problema
    vertices_nao_visitados = set(grafo.index)
    percurso = [vertice_inicial]
    vertices_nao_visitados.remove(vertice_inicial - 1)
    dist_total = 0
    v_atual = vertice_inicial
    while len(vertices_nao_visitados) > 0:
        # Obtem o label (numero) do vertice com menor custo
        idx_vizinho_mais_proximo = grafo[v_atual].loc[list(vertices_nao_visitados)].idxmin(skipna=True)
        prox = idx_vizinho_mais_proximo + 1
        dist_total += grafo[v_atual][idx_vizinho_mais_proximo]
        percurso.append(int(prox))
        v_atual = prox
        vertices_nao_visitados.remove(idx_vizinho_mais_proximo)
    dist_total += grafo[v_atual][vertice_inicial -1]
    percurso.append(vertice_inicial)
    return percurso, dist_total


## Aplicando nos problemas

In [9]:
def aplicar_problema(grafo, inicial, heuristica_construtiva, heuristicas_busca_local, out = None):
    caminho, custo = heuristica_construtiva(grafo, inicial)
    melhor_custo = custo
    melhor_caminho = caminho
    heuristica_melhor_solucao = None
    out = True if out is None else out
    if out:
        print("Aplicando Heuristica Construtiva: ", heuristica_construtiva.__name__)
        print(f"Caminho: {caminho}")
        print(f"Custo: {custo}")
        print("Aplicando busca(s) local(is):")
    for h in heuristicas_busca_local:
        solucao_local, custo_local = busca_local_primeira_melhoria(melhor_caminho, melhor_custo, h, grafo)
        if out:
            print(f"Heuristica : {h.__name__}")
            print("Tipo : Primeira Melhoria")
            print(f"Melhor caminho pos busca local: {solucao_local}")
            print(f"Melhor custo pos busca local: {custo_local}")
        if custo_local < melhor_custo:
            melhor_caminho = solucao_local
            melhor_custo = custo_local
            heuristica_melhor_solucao = h.__name__
    return melhor_caminho, melhor_custo, heuristica_melhor_solucao


### Problema 1
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pela distancia em
km.


In [10]:
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km, 1, vizinho_mais_proximo, [swap, shift, inversao])
print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")



Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Custo: 2291.3999999999996
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Melhor custo pos busca local: 2291.3999999999996
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Melhor custo pos busca local: 2291.3999999999996
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos bus


### Problema 2
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pelo tempo em minutos.


In [11]:
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min, 1, vizinho_mais_proximo, [swap, shift, inversao])
print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")


Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1]
Custo: 2355.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1]
Melhor custo pos busca local: 2355.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1]
Melhor custo pos busca local: 2355.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 2, 3, 4, 41, 30, 6

### Problema 3
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a distância
em km

In [12]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_km_36 = matriz_problema_km[list(range(1,37))].iloc[0:36].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_36, 1, vizinho_mais_proximo, [swap, shift, inversao])
print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")


Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 25, 17, 7, 19, 16, 34, 33, 20, 31, 36, 32, 26, 35, 13, 24, 29, 14, 5, 28, 18, 15, 6, 27, 30, 1]
Custo: 1951.4999999999998
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 25, 17, 7, 19, 16, 34, 33, 20, 31, 36, 32, 26, 35, 13, 24, 29, 14, 5, 28, 18, 15, 6, 27, 30, 1]
Melhor custo pos busca local: 1951.4999999999998
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 25, 17, 7, 19, 16, 34, 33, 20, 31, 36, 32, 26, 35, 13, 24, 29, 14, 5, 28, 18, 15, 6, 27, 30, 1]
Melhor custo pos busca local: 1951.4999999999998
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 4, 3, 2, 11, 10, 12, 9, 8, 21, 22, 36, 25, 31, 7, 19, 17, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 5, 18, 15, 28, 27, 6, 30

### Problema 4
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a o tempo de trajeto
em minutos.


In [13]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_min_36 = matriz_problema_min[list(range(1,37))].iloc[0:36].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_36, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")



Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 25, 17, 20, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 6, 30, 27, 7, 31, 36, 1]
Custo: 1950.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 22, 36, 31, 7, 12, 10, 11, 5, 2, 3, 4, 23, 21, 25, 17, 20, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 6, 30, 27, 1]
Melhor custo pos busca local: 1864.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 22, 36, 31, 7, 12, 10, 11, 5, 2, 3, 4, 23, 21, 25, 17, 20, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 6, 30, 27, 1]
Melhor custo pos busca local: 1864.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 4, 3, 2, 5, 11, 10, 12, 9, 8, 21, 22, 36, 25, 31, 7, 17, 20, 33, 19, 34, 16, 32, 26, 35, 13, 24, 29, 14, 18, 15, 28, 30, 6, 27, 1]
Melhor custo pos busca local: 1

### Problema 5
Percurso por 24 cidades, partindo de ANGICOS, tomando como função custo a distância em km


In [33]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 24]
matriz_problema_km_24 = matriz_problema_km[list(range(1,25))].iloc[0:24].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_24, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 1]
Custo: 1379.3
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 1]
Melhor custo pos busca local: 1379.3
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 1]
Melhor custo pos busca local: 1379.3
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 4, 3, 2, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 1]
Melhor custo pos busca local: 1343.3000000000002

Melhor caminho: [1, 23, 4, 3, 2, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 1]
Melhor custo: 1343.3000000000002
Heuristica: inversao


### Problema 6
Percurso por 24 cidades, partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos



In [35]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 24]
matriz_problema_min_24 = matriz_problema_min[list(range(1,25))].iloc[0:24].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_24, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 17, 20, 19, 16, 7, 13, 24, 14, 18, 15, 6, 1]
Custo: 1283.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 17, 20, 19, 16, 7, 13, 24, 14, 18, 15, 6, 1]
Melhor custo pos busca local: 1283.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 17, 20, 19, 16, 7, 13, 24, 14, 18, 15, 6, 1]
Melhor custo pos busca local: 1283.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 4, 3, 2, 5, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 18, 15, 6, 1]
Melhor custo pos busca local: 1248.0

Melhor caminho: [1, 23, 4, 3, 2, 5, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 18, 15, 6, 1]
Melhor custo: 1248.0
Heuristica: inversao


### Problema 7
Percurso por 12 cidades, partindo de ANGICOS, tomando como função custo a distância em km

In [36]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 12]
matriz_problema_km_12 = matriz_problema_km[list(range(1,13))].iloc[0:12].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_12, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 10, 12, 11, 2, 3, 4, 5, 7, 6, 1]
Custo: 828.4
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, 2, 11, 10, 7, 5, 4, 6, 8, 9, 12, 1]
Melhor custo pos busca local: 793.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, 2, 11, 10, 7, 5, 4, 6, 8, 9, 12, 1]
Melhor custo pos busca local: 793.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 6, 4, 3, 2, 11, 5, 7, 9, 8, 10, 12, 1]
Melhor custo pos busca local: 714.0

Melhor caminho: [1, 6, 4, 3, 2, 11, 5, 7, 9, 8, 10, 12, 1]
Melhor custo: 714.0
Heuristica: inversao


### Problema 8
Percurso por 12 cidades, partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [38]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 12]
matriz_problema_min_12 = matriz_problema_min[list(range(1,13))].iloc[0:12].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_12, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 6, 7, 1]
Custo: 687.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 609.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 609.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 609.0

Melhor caminho: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Melhor custo: 609.0
Heuristica: swap


### Problema 9
Percurso por 7 cidades (1, 7, 8, 9, 10, 11, 12), com função custo em km


In [40]:
cidades_7 = [1, 7, 8, 9, 10, 11, 12]
# Converte IDs (1-based) para índices de linha (0-based)
indices_7 = [c - 1 for c in cidades_7]

# Seleciona as colunas (IDs) e as linhas (índices) corretas
matriz_problema_km_7 = matriz_problema_km[cidades_7].iloc[indices_7].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_7, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 10, 12, 11, 7, 1]
Custo: 518.5
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 11, 10, 8, 9, 12, 7, 1]
Melhor custo pos busca local: 459.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 11, 10, 8, 9, 12, 7, 1]
Melhor custo pos busca local: 459.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 11, 10, 8, 9, 12, 7, 1]
Melhor custo pos busca local: 459.0

Melhor caminho: [1, 11, 10, 8, 9, 12, 7, 1]
Melhor custo: 459.0
Heuristica: swap


### Problema 10
Percurso por 7 cidades (1, 7, 8, 9, 10, 11, 12), com função custo em minutos


In [42]:
cidades_7 = [1, 7, 8, 9, 10, 11, 12]
# Converte IDs (1-based) para índices de linha (0-based)
indices_7 = [c - 1 for c in cidades_7]

# Seleciona as colunas (IDs) e as linhas (índices) corretas
matriz_problema_min_7 = matriz_problema_min[cidades_7].iloc[indices_7].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_7, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 12, 10, 11, 7, 1]
Custo: 413.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 12, 7, 9, 10, 11, 1]
Melhor custo pos busca local: 399.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 12, 7, 9, 8, 10, 11, 1]
Melhor custo pos busca local: 392.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 7, 12, 10, 11, 1]
Melhor custo pos busca local: 364.0

Melhor caminho: [1, 8, 9, 7, 12, 10, 11, 1]
Melhor custo: 364.0
Heuristica: inversao


### Problema 11
Percurso por 6 cidades (1 a 6), partindo de ANGICOS, tomando como função custo a distância em km


In [44]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 6]
matriz_problema_km_6 = matriz_problema_km[list(range(1,7))].iloc[0:6].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_6, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 3, 4, 2, 5, 6, 1]
Custo: 463.6
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 6, 4, 3, 2, 5, 1]
Melhor custo pos busca local: 348.3
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 6, 4, 3, 2, 5, 1]
Melhor custo pos busca local: 348.3
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 6, 4, 3, 2, 5, 1]
Melhor custo pos busca local: 348.3

Melhor caminho: [1, 6, 4, 3, 2, 5, 1]
Melhor custo: 348.3
Heuristica: swap


### Problema 12
Percurso por 6 cidades (1 a 6), partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [45]:

# Filtra as colunas e linhas do DataFrame para as cidades [1, 6]
matriz_problema_min_6 = matriz_problema_min[list(range(1,7))].iloc[0:6].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_6, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 3, 4, 2, 5, 6, 1]
Custo: 400.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 305.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 305.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 305.0

Melhor caminho: [1, 5, 2, 3, 4, 6, 1]
Melhor custo: 305.0
Heuristica: swap


### Insercao Mais Barata

In [15]:
import math

# Estamos considerando o label numerico
def insercao_mais_barata(grafo : pd.DataFrame, vertice_inicial : int | str):

    vertices_restantes = set(grafo.index)

    # Selecionamos o indice do vertice mais proximo a raiz de forma gulosa
    v_mais_proximo = grafo[vertice_inicial].idxmin()

    # Ciclo inicial
    rota = [vertice_inicial, v_mais_proximo + 1, vertice_inicial]
    vertices_restantes.remove( vertice_inicial - 1, )
    vertices_restantes.remove(v_mais_proximo)
    vertices_inseridos = {vertice_inicial - 1, v_mais_proximo}

    custo_total = 0
    while vertices_restantes:
        # Estamos selecionando o vertice de 'vertices_restantes' que tem a menor distancia a outro vertice
        # Filtra apenas as linhas da matriz referentes aos vertices do conjunto de vertices_restantes
        r = grafo[list(vertices_restantes)].iloc[list(vertices_inseridos)].min().idxmin()

        # Consegue o indice do vertice do conjunto 'vertices_restantes' que tem a menor distancia
        # ao algum vertice incluido na rota. Para manter este codigo similar ao pseudocodigo visto em sala, chamaremos a variavel de
        # Encontrar a melhor posicao para inserir a rota
        melhor_custo_insercao = math.inf
        melhor_posicao_insercao = None
        for i in range(1, len(rota) - 1):
            u, v = rota[i] - 1, rota[i + 1] - 1
            custo = grafo[u + 1][r] + grafo[r + 1][v] - grafo[u + 1][v]
            if custo < melhor_custo_insercao:
                melhor_custo_insercao = custo
                melhor_posicao_insercao = i

        rota.insert(melhor_posicao_insercao + 1, r + 1)
        vertices_restantes.remove(r)
        vertices_inseridos.add(r)
        custo_total += melhor_custo_insercao
    return rota, calcular_custo(rota, grafo)



### Problema 1
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pela distancia em
km.


In [16]:
melhor_custo, melhor_caminho, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km, 1, insercao_mais_barata, [swap, shift, inversao])

print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")


Aplicando Heuristica Construtiva:  insercao_mais_barata
Caminho: [1, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 23, 1]
Custo: 2148.1
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 1]
Melhor custo pos busca local: 2148.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 21, 8, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 1]
Melhor custo pos busca local: 2147.2
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 2, 11, 5, 46, 14, 43, 

## Algoritmo Genetico

### Gerador de solucoes aleatorias

In [17]:
import random
def solucao_aleatoria(grafo : pd.DataFrame, vertice_inicial):
    rota = list(map(lambda i : i + 1,grafo.index))
    rota.remove(vertice_inicial)
    random.shuffle(rota)
    return [vertice_inicial] + rota + [vertice_inicial]

# Funca para obter o custo total a partir de uma solucao (caminho)
def calcular_custo(solucao, grafo):
    if len(solucao) <= 1:
        return
    custo = 0
    for prox in range(1, len(solucao)):
        v1 = solucao[prox - 1]
        v2 = solucao[prox]
        # print("v1: ", v1)
        # print("v2: ", v2)
        custo += grafo[v1][v2 - 1]
    return custo

samples = [solucao_aleatoria(matriz_problema_km, 1) for i in range(0,6)]
print(samples)
# custos = [calcular_custo(solucao, matriz_problema_km) for solucao in samples]
# print(f"Minimo : {min(custos)}")
# print(f"Maximo : {max(custos)}")
#

[[1, 16, 34, 13, 43, 45, 24, 19, 30, 22, 9, 39, 20, 11, 10, 3, 38, 7, 27, 4, 29, 37, 6, 2, 36, 21, 23, 35, 42, 17, 14, 41, 44, 28, 32, 26, 25, 12, 46, 15, 33, 8, 40, 48, 18, 5, 31, 47, 1], [1, 31, 3, 42, 5, 6, 9, 21, 15, 26, 41, 39, 12, 14, 36, 4, 24, 7, 27, 46, 22, 45, 38, 29, 13, 30, 33, 37, 43, 23, 2, 47, 17, 44, 48, 40, 25, 32, 10, 34, 11, 18, 8, 16, 19, 35, 20, 28, 1], [1, 43, 35, 42, 19, 40, 47, 8, 38, 30, 3, 18, 22, 2, 33, 44, 17, 32, 46, 24, 34, 13, 11, 6, 12, 28, 37, 48, 45, 27, 23, 26, 7, 39, 15, 16, 14, 4, 31, 20, 9, 21, 10, 36, 5, 29, 41, 25, 1], [1, 4, 20, 18, 7, 11, 43, 9, 44, 34, 27, 37, 28, 30, 5, 21, 47, 33, 42, 29, 15, 36, 14, 16, 10, 23, 25, 39, 45, 38, 3, 40, 13, 8, 19, 6, 2, 24, 35, 48, 12, 41, 32, 22, 26, 31, 46, 17, 1], [1, 16, 44, 8, 45, 13, 26, 7, 5, 15, 41, 2, 34, 17, 25, 32, 39, 37, 22, 12, 21, 43, 24, 47, 38, 9, 29, 46, 48, 11, 20, 36, 30, 27, 3, 35, 18, 4, 40, 23, 33, 19, 10, 42, 6, 28, 31, 14, 1], [1, 48, 25, 26, 43, 11, 33, 46, 7, 12, 34, 18, 6, 32, 30, 3

## Gerando populacao
Gerar 15 indivíduos

### Populacao para o problema 1

In [18]:
solucao_insercao_mais_barata = insercao_mais_barata(matriz_problema_km, 1)[0]
solucao_vizinho_mais_proximo = vizinho_mais_proximo(matriz_problema_km, 1)[0]
solucao_insercao_mais_barata_local = aplicar_problema(matriz_problema_km, 1, insercao_mais_barata, [swap, shift, inversao], out=False)[0]
solucao_vizinho_mais_proximo_local = aplicar_problema(matriz_problema_km, 1, vizinho_mais_proximo, [swap, shift, inversao], out=False)[0]

solucoes_aleatorias = [solucao_aleatoria(matriz_problema_km, 1) for i in range(0,36)]

samples = solucoes_aleatorias + [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local]
# samples = [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local]

print(samples)
# Ordenando pra facilitar a selecao por elitismo
samples = sorted(samples, key=lambda solucao: calcular_custo(solucao, matriz_problema_km))


[[1, 41, 18, 17, 37, 22, 25, 16, 27, 13, 28, 48, 39, 7, 26, 11, 33, 19, 24, 2, 9, 46, 20, 45, 36, 5, 6, 34, 3, 47, 38, 29, 12, 4, 30, 44, 8, 31, 35, 21, 10, 32, 42, 40, 43, 23, 15, 14, 1], [1, 4, 2, 28, 22, 7, 25, 27, 40, 26, 6, 14, 21, 38, 36, 20, 45, 13, 15, 11, 41, 5, 43, 35, 10, 31, 23, 29, 24, 34, 9, 32, 12, 44, 17, 16, 37, 3, 42, 46, 39, 19, 18, 8, 30, 48, 33, 47, 1], [1, 28, 7, 3, 48, 33, 15, 20, 12, 18, 39, 36, 19, 22, 24, 45, 14, 5, 40, 35, 43, 10, 17, 44, 38, 42, 23, 8, 26, 25, 27, 29, 32, 11, 2, 34, 21, 30, 47, 46, 37, 41, 4, 9, 13, 6, 31, 16, 1], [1, 46, 16, 21, 33, 42, 13, 45, 32, 28, 6, 48, 34, 7, 41, 17, 20, 30, 43, 14, 40, 5, 29, 26, 31, 9, 18, 12, 11, 25, 15, 10, 36, 2, 24, 44, 47, 8, 35, 37, 38, 3, 27, 23, 39, 22, 19, 4, 1], [1, 30, 46, 27, 36, 48, 39, 3, 31, 34, 25, 8, 18, 38, 13, 35, 47, 29, 40, 12, 16, 4, 20, 11, 17, 41, 6, 22, 26, 32, 42, 10, 5, 37, 23, 7, 24, 28, 14, 19, 45, 33, 43, 9, 21, 2, 44, 15, 1], [1, 13, 3, 17, 21, 35, 7, 11, 36, 26, 46, 16, 32, 42, 25, 2

## 1) Inicio
Organizando os cromossomos

In [19]:
cromossomos = [(solucao, calcular_custo(solucao, matriz_problema_km)) for solucao in samples]


## Metodos de Selecao

In [20]:
def elitismo(populacao : list, num_pares):
    populacao_ordenada = sorted(populacao, key=lambda x: x[1])
    genitores = []
    for i in range(0,num_pares):
        genitores.append((populacao_ordenada[i], populacao_ordenada[i + 1]))
    return genitores

def torneio(populacao: list, num_pares: int, tamanho_torneio: int = 3):
  genitores = []
  for _ in range(num_pares):
    competidores_1 = random.sample(populacao, k=tamanho_torneio)
    pai1 = min(competidores_1, key=lambda x: x[1])
    competidores_2 = random.sample(populacao, k=tamanho_torneio)
    pai2 = min(competidores_2, key=lambda x: x[1])
    genitores.append((pai1, pai2))
  return genitores



### Metodos de Cruzamento

In [21]:
def crossover_1_ponto(cromossomo_1, cromossomo_2):
    n = len(cromossomo_1) # Cromossomos tem comprimentos iguais
    # Ponto de corte
    mid = n // 2
    offspring_1 = cromossomo_1[0:mid] + cromossomo_2[mid:n]
    offspring_2 = cromossomo_2[0:mid] + cromossomo_1[mid:n]
    offsprings = [offspring_1, offspring_2]
    # As labels sao numeros em sequencia
    vertices = set(range(2, n))
    for i, offspring in enumerate(offsprings):
        # vertices ja presentes no offspring atual
        vertices_no_offspring = set(offspring)
        vertices_no_offspring.remove(1)
        # se o numero de elementos unicos eh menor que o tamanho - 1 (ja que eh um ciclo),
        # ja elementos repetidos
        if len(vertices_no_offspring) < n-1:
            vertices_restantes = vertices.difference(vertices_no_offspring)
            novo_offspring = offspring[0:mid]
            controle = set(offspring[1:mid])
            for v in offspring[mid:]:
                novo_v = None
                if v in controle:
                    novo_v = vertices_no_offspring.pop()
                else:
                    novo_v = v
                controle.add(novo_v)
                novo_offspring.append(novo_v)
            offsprings[i] = novo_offspring
    return offsprings[0], offsprings[1]

def crossover_ox(genitor_1, genitor_2):
    offspring_1, offspring_2 = genitor_1[:len(genitor_1) // 2], genitor_2[:len(genitor_2) // 2]
    vertices_incluidos1, vertices_incluidos2 = set(offspring_1), set(offspring_2)
    for i in genitor_2:
        if i not in vertices_incluidos1:
            offspring_1.append(i)
    offspring_1.append(genitor_1[0])
    for i in genitor_1:
        if i not in vertices_incluidos2:
            offspring_2.append(i)
    offspring_2.append(genitor_2[0])
    return offspring_1, offspring_2

def crossover_2_pontos(genitor_1, genitor_2):
    offspring_1, offspring_2 = genitor_1[:len(genitor_1) // 2], genitor_2[:len(genitor_2) // 2]
    vertices_incluidos1, vertices_incluidos2 = set(offspring_1), set(offspring_2)
    g1, g2 = random.sample(range(1,len(genitor_1) - 1), 2)
    g_1_inicio, g_1_final = min(g1,g2), max(g1,g2)
    g3, g4 = random.sample(range(1,len(genitor_1) - 1), 2)
    g_2_inicio, g_2_final = min(g1,g2), max(g1,g2)
    for i in genitor_2:
        if i not in vertices_incluidos1:
            offspring_1.append(i)
    offspring_1.append(genitor_1[0])
    for i in genitor_1:
        if i not in vertices_incluidos2:
            offspring_2.append(i)
    offspring_2.append(genitor_2[0])
    return offspring_1, offspring_2
    pass

def crossover_uniforme():
    pass

def cruzamento(genitores, grafo):
    offsprings = []
    for g in genitores:
        genitor_1,genitor_2=g
        offspring1, offspring2 = crossover_ox(genitor_1[0], genitor_2[0])
        offsprings.extend([(offspring1, calcular_custo(offspring1, grafo)), (offspring2,calcular_custo(offspring1, grafo))])
    return offsprings

In [22]:
import random
def mutacao_swap(cromossomos, probabilidade, grafo):
    novos_cromossomos = []
    for cromossomo in cromossomos:
        cromossomo = list(cromossomo[0]) # extrai o caminho

        if random.random() < probabilidade:
            gene_1, gene_2 = random.sample(range(1, len(cromossomo) - 1), 2)
            cromossomo[gene_1], cromossomo[gene_2] = cromossomo[gene_2], cromossomo[gene_1]
        # Adiciona o cromossomo (mutado ou não) com seu novo custo
        novos_cromossomos.append((cromossomo, calcular_custo(cromossomo, grafo)))

    return novos_cromossomos

def mutacao_inversao(cromossomos, probabilidade, grafo):
    novos_cromossomos = []
    for cromossomo_original_tuple in cromossomos:
        cromossomo = list(cromossomo_original_tuple[0]) # Trabalhar com a lista da rota

        if random.random() < probabilidade:
            gene_1, gene_2 = random.sample(range(1, len(cromossomo) - 1), 2)
            gene_inicio = min(gene_1, gene_2)
            gene_fim = max(gene_1, gene_2)
            parte_invertida = cromossomo[gene_inicio:gene_fim][::-1]
            novo = cromossomo[:gene_inicio] + parte_invertida + cromossomo[gene_fim:]

            novos_cromossomos.append((novo, calcular_custo(novo,grafo)))
            # print(f"Custo mutado : {calcular_custo(novo, grafo)}")
        else:
            novos_cromossomos.append(cromossomo_original_tuple)
        # Adiciona o cromossomo (mutado ou não) com seu novo custo

    return novos_cromossomos




In [23]:
# for custo, cromossomo in cromossomos.items():
#     mutacao_swap(cromossomo, 0.5)

In [24]:
a = {1:"a"}
b = random.choice(list(a.items()))
b

(1, 'a')

In [25]:
def renovacao(populacao_atual, offspring, num_populacao):
    pop = populacao_atual + offspring
    # offspring_ordenada = sorted(offspring, key=lambda a : a[1])
    # pop_ordenada = sorted(populacao_atual, key=lambda a : a[1])
    pop_ordenada = sorted(pop, key=lambda x: x[1])
    return pop_ordenada[:num_populacao]
    # Usando torneio
    # nova_populacao = []
    # pop = populacao_atual + offspring
    # while len(nova_populacao) < num_populacao:
    #     candidatos = random.choices(pop, k=3)
    #     # offspring_individuo = random.choice(list(offspring))
    #     escolhido = min(candidatos, key=lambda x: x[1])
    #     nova_populacao.append(escolhido)
    # return nova_populacao

def roleta(populacao, offspring, num_populacao):
    populacao = populacao + offspring
    cromossomos = []
    somatorio_custo = sum(map(lambda cromossomo : cromossomo[1], populacao))
    values = [solucao[1]/somatorio_custo for solucao in populacao]
    fitness = []
    for val in values:
        min_val = min(values)
        max_val = max(values)
        normalized_data = (max_val - val) / (max_val - min_val)
        fitness.append(normalized_data)
    print(f"Custos : ", list(map(lambda x : x[1], populacao)))
    print(f"Fitness : ", fitness)
    solucoes = list(map(lambda c : c[1], populacao))
    return random.choices(populacao, weights=fitness, k=num_populacao)



In [26]:

print(float(random.random()))

0.028992383426935775


def algoritmo_genetico(numero_iteracoes, taxa_mutacao, mutacoes, selecao,

In [27]:
def algoritmo_genetico(grafo, populacao_inicial, numero_iteracoes, taxa_mutacao, mutacao, selecao, cruzamento, renovacao):
    populacao_atual = populacao_inicial
    for iter in range(0, numero_iteracoes):
        genitores = selecao(populacao_atual, len(populacao_atual) // 2)
        # print(f"Numero de genitores: {len(genitores)}")
        offsprings = cruzamento(genitores, grafo)
        # Ja aplica a taxa de mutacao a populacao
        offsprings = mutacao(offsprings, taxa_mutacao, grafo)
        # print(f"Tamanho do offspring: {len(offsprings)}")
        # print(offsprings)
        populacao_atual = renovacao(populacao_atual, offsprings, len(populacao_atual))
        # print(f"Tamanho da pop: {len(populacao_atual)}")
    return populacao_atual

custos = []
for i in range(10):
    resultado = algoritmo_genetico(matriz_problema_km, cromossomos, 100, 0.4, mutacao_inversao, elitismo, cruzamento, renovacao)
    custos.append(min(resultado, key=lambda x : x[1])[1])
print(custos)

[np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997), np.float64(2084.2999999999997)]


## Memetico

In [28]:
import random

def algoritmo_memetico(grafo, populacao_inicial, numero_iteracoes, taxa_mutacao,
                       mutacao, selecao, cruzamento, renovacao,
                       heuristicas_busca_local, probabilidade_busca_local=0.2):
    populacao_atual = populacao_inicial

    for iteracao in range(numero_iteracoes):

        n_genitores = len(populacao_atual) // 2
        genitores = selecao(populacao_atual, n_genitores)
        print(f"Numero de genitores: {len(genitores)}")
        offsprings = cruzamento(genitores, grafo)
        offsprings = mutacao(offsprings, taxa_mutacao, grafo)

        novos_offsprings = []

        for item in offsprings:
            if isinstance(item, tuple):
                individuo = item[0]
            else:
                individuo = item

            custo_individuo = calcular_custo(individuo, grafo)

            if random.random() < probabilidade_busca_local:
                melhorou = False
                for heuristica in heuristicas_busca_local:
                    nova_rota, novo_custo = busca_local_primeira_melhoria(
                        individuo, custo_individuo, heuristica, grafo
                    )

                    if novo_custo < custo_individuo:
                        individuo = nova_rota
                        custo_individuo = novo_custo
                        melhorou = True

            novos_offsprings.append((individuo, custo_individuo))

        print(f"Tamanho do offspring: {len(novos_offsprings)}")
        print(novos_offsprings)
        print(f"Tamanho da pop: {len(populacao_atual)}")

        populacao_atual = renovacao(populacao_atual, novos_offsprings, len(populacao_atual))

        melhor_custo_geracao = populacao_atual[0][1]
        print(f"Geração {iteracao + 1}/{numero_iteracoes} | Melhor Custo: {melhor_custo_geracao:.2f} | Pop: {len(populacao_atual)}")

    return populacao_atual

In [29]:
resultado = algoritmo_memetico(
    grafo=matriz_problema_km,
    populacao_inicial=cromossomos,
    numero_iteracoes=20,
    taxa_mutacao=0.3,
    mutacao=mutacao_inversao,
    selecao=elitismo,
    cruzamento=cruzamento,
    renovacao=renovacao,
    heuristicas_busca_local=[swap, shift, inversao]
)

print(resultado)

Numero de genitores: 20
Tamanho do offspring: 40
[([1, 2, 11, 5, 46, 14, 43, 29, 24, 13, 47, 37, 38, 20, 33, 39, 35, 42, 26, 32, 16, 34, 19, 7, 23, 4, 3, 10, 12, 9, 8, 21, 22, 36, 48, 40, 25, 31, 17, 41, 30, 6, 27, 44, 28, 15, 18, 45, 1], np.float64(2510.9)), ([1, 23, 4, 3, 10, 11, 2, 12, 9, 8, 21, 22, 36, 48, 40, 25, 31, 7, 19, 17, 20, 33, 16, 34, 5, 46, 14, 43, 29, 24, 13, 47, 37, 38, 39, 35, 42, 26, 32, 30, 6, 27, 28, 15, 18, 45, 44, 41, 1], np.float64(2580.1)), ([1, 3, 4, 23, 8, 12, 10, 11, 2, 5, 46, 14, 43, 29, 24, 13, 37, 47, 39, 35, 42, 26, 32, 34, 16, 33, 19, 7, 31, 17, 20, 38, 25, 40, 48, 36, 22, 9, 21, 30, 6, 27, 28, 15, 18, 45, 44, 41, 1], np.float64(2078.0)), ([1, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 23, 4, 3, 2, 20, 47, 37, 38, 13, 24, 29, 43, 14, 46, 5, 41, 30, 6, 27, 44, 28, 15, 18, 45, 1], np.float64(2570.7999999999997)), ([1, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 2, 3